In [1]:
import pandas as pd

In [2]:
fashion = pd.read_csv("./fashion.csv", encoding="utf-8")
cosmetic = pd.read_csv("./cosmetic.csv", encoding="utf-8")
digital = pd.read_csv("./digital.csv", encoding="utf-8")
interior = pd.read_csv("./interior.csv", encoding="utf-8")
kid = pd.read_csv("./kid.csv", encoding="utf-8")
food = pd.read_csv("./food.csv", encoding="utf-8")
sports = pd.read_csv("./sports.csv", encoding="utf-8")
life = pd.read_csv("./life.csv", encoding="utf-8")
culture = pd.read_csv("./culture.csv", encoding="utf-8")
fashion.head()

,Unnamed: 0,name
0,0,1+1 여자 레깅스 필라테스복 요가복하의 바지
1,1,반팔 간호복 수술복 간호사복 유니폼 상의
2,2,단체티셔츠 30수유색반팔 교회선교티 단체티제작
3,3,탑앤탑 - 단체복 라운드티 카라티 후드티 제작
4,4,1+1 소매퍼프 티셔츠 4컬러


In [3]:
food.head()

,Unnamed: 0,name
0,0,[맛있닭] 닭가슴살 볼 큐브 외 6종 1kg
1,1,[맛있닭] 닭가슴살스테이크 외 29종 골라담기
2,2,"바날라 보존료, 발색제 없는 촉촉한 닭가슴살(네 가지 맛 선택) 100g"
3,3,아임닭 닭가슴살 모음 골라담기
4,4,족만이 순살족발


In [4]:
print(len(fashion["name"]), len(cosmetic["name"]), len(digital["name"]), len(interior["name"]), len(kid["name"]),
      len(food["name"]), len(sports["name"]), len(life["name"]), len(culture["name"]))

29215 27207 39445 33395 21000 23077 23071 20774 6000


In [5]:
fashion_df = pd.DataFrame(fashion["name"])
fashion_df["label"] = "fashion"
cosmetic_df = pd.DataFrame(cosmetic["name"])
cosmetic_df["label"] = "cosmetic"
digital_df = pd.DataFrame(digital["name"])
digital_df["label"] = "digital"
interior_df = pd.DataFrame(interior["name"])
interior_df["label"] = "interior"
kid_df = pd.DataFrame(kid["name"])
kid_df["label"] = "kid"
food_df = pd.DataFrame(food["name"])
food_df["label"] = "food"
sports_df = pd.DataFrame(sports["name"])
sports_df["label"] = "sports"
life_df = pd.DataFrame(life["name"])
life_df["label"] = "life"
culture_df = pd.DataFrame(culture["name"])
culture_df["label"] = "culture"

In [6]:
data = fashion_df.append(cosmetic_df).append(digital_df).append(interior_df).append(kid_df).append(food_df).append(sports_df).append(life_df).append(culture_df)

In [7]:
import numpy as np
from sklearn.model_selection import train_test_split

In [8]:
train, test = train_test_split(data, test_size=0.33, random_state = 42)

In [9]:
train["label"] = pd.Categorical(train["label"])

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [10]:
train.groupby("label").count()

,name
label,
cosmetic,18192
culture,4014
digital,26525
fashion,19699
food,15315
interior,22431
kid,14197
life,13833
sports,15327


In [11]:
test.groupby("label").count()

,name
label,
cosmetic,9015
culture,1986
digital,12920
fashion,9516
food,7762
interior,10964
kid,6803
life,6941
sports,7744


In [12]:
from konlpy.tag import Twitter
from sklearn.feature_extraction.text import CountVectorizer

In [13]:
def get_noun(text):
    tokenizer = Twitter()
    nouns = tokenizer.nouns(text)
    return [n for n in nouns]

In [14]:
cv = CountVectorizer(tokenizer=get_noun)

In [15]:
tdm = cv.fit_transform(train["name"])

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [16]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline

text_clf_svm = Pipeline([('vect', CountVectorizer(tokenizer=get_noun)), ('tfidf', TfidfTransformer()), ('clf-svm', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, n_iter=5, random_state=42))])

text_clf_svm = text_clf_svm.fit(train["name"], train["label"])

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:152: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


In [17]:
predicted_svm = text_clf_svm.predict(test["name"])
np.mean(predicted_svm == test["label"])

0.8429892330043041

In [18]:
text_clf_svm.predict(["묵은지"])

array(['food'], dtype='<U8')

In [21]:
text_clf_svm.predict(["삼성전자 티비"])

array(['digital'], dtype='<U8')

In [22]:
import pickle
from sklearn.externals import joblib 
# 객체를 pickled binary file 형태로 저장한다 
joblib.dump(text_clf_svm, "naver_shopping_model.pkl") 

['naver_shopping_model.pkl']